# Ejercicios Pair Programming - Módulo 2 Sprint 2
## Lección 5 - ETL 3.2
### Natalia García Lago y Noemi Azorín

Igual que en el ejercicio de pair programming de ETL Transformación II, tendréis que crear una clase que nos permita cargar los datos en SQL I.